In [1]:
import sys

sys.path.append("../../")
from config.get_config import get_config

In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
import shap
import numpy as np
from random import randint

from sklearn.metrics import mean_squared_error

In [3]:
sys.path.append("../../src")
from pipeline.factory import Factory
from preprocessing.process_raw_data import load_data
from models.training_helpers import generate_ml_features

config = dict(get_config())
data_config = dict(config["data"])

mlflow.set_tracking_uri("http://0.0.0.0:5000")
e = mlflow.set_experiment(data_config["mlflow_experiment_name"])

2024-02-19 16:11:01.893821: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-19 16:11:02.008610: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 16:11:02.008703: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 16:11:02.008984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 16:11:02.084494: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-19 16:11:02.087226: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
# fix paths
for k, v in data_config["paths"].items():
    if k == "scaler" or k == "optim":
        continue
    for k2, v2 in v.items():
        data_config["paths"][k][k2] = "../../" + v2

In [5]:
df_or = load_data(data_config["paths"])
df_or.head()

,date,store_nbr,family,sales,onpromotion,dcoilwtico,typedays,locale,locale_name,description,transferred,city,state,typestores,cluster
0,2013-01-01,1,AUTOMOTIVE,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13


In [6]:
f = Factory(data_config)
f.fit(df_or)

train, val = f.get_df()
train.head()

Fitting data ...



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,onpromotion,dcoilwtico,sales,typedays_Additional,typedays_Event,typedays_Holiday,typedays_Normal
2013-01-01,0.0,0.792965,2511.618999,False,False,True,False
2013-01-02,0.0,0.792965,496092.417944,False,False,False,True
2013-01-03,0.0,0.790951,361461.231124,False,False,False,True
2013-01-04,0.0,0.792728,354459.677093,False,False,False,True
2013-01-05,0.0,0.793044,348347.385084,False,False,False,True


In [9]:
generate_ml_features(train)[0][["month", "day_of_month", "day_of_year", "week_of_year", "week_of_month", "day_of_week", "is_year_start", "is_year_end", "season", "is_month_end", "is_month_start", "is_wknd", "year"]].head()

,month,day_of_month,day_of_year,week_of_year,week_of_month,day_of_week,is_year_start,is_year_end,season,is_month_end,is_month_start,is_wknd,year
2013-01-01,1,1,1,1,1,2,1,0,0,0,1,0,2013
2013-01-02,1,2,2,1,1,3,0,0,0,0,0,0,2013
2013-01-03,1,3,3,1,1,4,0,0,0,0,0,0,2013
2013-01-04,1,4,4,1,1,5,0,0,0,0,0,1,2013
2013-01-05,1,5,5,1,1,6,0,0,0,0,0,1,2013


In [8]:
h = pd.read_csv(f'../../{data_config["paths"]["raw"]["holidays"]}')
h[h["type"] == "Event"]

FileNotFoundError: [Errno 2] No such file or directory: '../../../../data/raw/holidays_events.csv'

In [ ]:
data_config["paths"]["raw"]